In [1]:
import PAA_LISA
import NOISE_LISA
import os

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import random
import os
from fractions import Fraction
import math
import datetime
from scipy.interpolate import interp1d
from scipy.interpolate import RegularGridInterpolator
import warnings
warnings.filterwarnings('ignore', 'The iteration is not making good progress')
#warnings.filterwarnings("error")
import scipy.optimize

year2sec=32536000
day2sec=year2sec/365.25
c=300000000

In [3]:
input_param = {
        'calc_method': 'Waluschka',
        'plot_on':False, #If plots will be made
        'dir_savefig': os.getcwd(), # The directory where the figures will be saved. If False, it will be in the current working directory
        'noise_check':False,
        'home':'/home/ester/git/synthlisa/', # Home directory
        'directory_imp': False,
        'num_back': 0,
        'dir_orbits': '/home/ester/git/synthlisa/orbits/', # Folder with orbit files
        'length_calc': 30, # Length of number of imported datapoints of orbit files. 'all' is also possible
        'dir_extr': 'zzzWaluschka_no_abberation', # This will be added to the folder name of the figures
        'timeunit':'Default', # The timeunit of the plots (['minutes'],['days']['years'])
        'LISA_opt':True, # If a LISA object from syntheticLISA will be used for further calculations (not sure if it works properly if this False)
        'arm_influence': True, # Set True to consider the travel time of the photons when calculating the nominal armlengths
        'tstep':False,
        'delay':True, #'Not ahead' or False
        'method':'fsolve', # Method used to solve the equation for the photon traveling time
        'valorfunc':'Function', #
        'select':'Hallion', # Select which orbit files will be imported ('all' is all)
        'test_calc':False,
        'abberation':False,
        'delay': True
        }

data_all = PAA_LISA.runfile.do_run(input_param)

['positions_timestep_days_scale_1000.txt', 'Folkner_orbit_timestep_seconds_scale_10.txt', 'readable.py', 'NGO_1M_10deg_synthlisa_timestep_days_scale_1000.txt']
['McKenzie_orbit_timestep_seconds_scale_1000.txt', 'Folkner_orbit_timestep_seconds_scale_1.txt', 'Hallion_2pt5G_orbits_pos_uniquedays_timestep_days_scale_1000.txt']
['Folkner_orbit_timestep_seconds_scale_10.txt', 'NGO_1M_10deg_synthlisa_timestep_days_scale_1.txt', 'Folkner_orbit_timestep_seconds_scale_1.txt', 'NGO_1M_10deg_synthlisa_timestep_days_scale_1000.txt']
Dir_extr:zzzWaluschka_no_abberation
Dir_extr:zzzWaluschka_no_abberation
Dir_extr:zzzWaluschka_no_abberation
Dir_extr:zzzWaluschka_no_abberation
Dir_extr:zzzWaluschka_no_abberation
Dir_extr:zzzWaluschka_no_abberation
Getting scale by filename:
1000.0

Getting timestep by filename:
days

Waluschka

Importing Orbit
30 datapoints
Done in 0.051176
Selected calculation method is: Waluschka

Selected calculation method is: Waluschka

Selected calculation method is: Waluschka



In [4]:
for k in range(0,len(data_all)/2):
    #data = PAA_res[str(k+1)]
    data = data_all[str(k+1)]
t_vec = data.t_all

In [5]:
Ndata = NOISE_LISA.Noise(data=data)

Obtaining noise
Obaining lasernoise
Obtaining shotnoise


In [6]:
w0 = Ndata.data.w0
labda = Ndata.data.labda

In [7]:
def w(z,w0,labda):
    zR = np.pi*(w0**2)/labda
    
    return w0*((1+((z/zR)**2))**0.5)

def R(z,w0,labda,guess=False):
    zR = np.pi*(w0**2)/labda
    
    if guess==False:
        return abs(z*(1+((zR/z)**2)))
    
    elif guess==True:
        return z

def phase_extra(point,w0,labda,Z=False):
    guess_set=False
    [xp,yp,zp] = point
    xp = np.float64(xp)
    yp = np.float64(yp)
    zp = np.float64(zp)
    if Z==False:
        Z=zp
    
    rp = (xp**2+yp**2)**0.5
    R_dz = lambda dz: R(dz+zp,w0,labda,guess=guess_set)
    f_solve = lambda dz: (R_dz(dz) - (R_dz(dz)**2- rp**2)**0.5) - dz
    
    
    #f_solve = lambda dz: R(dz+zp,w0,labda,guess=guess_set)**2 - rp**2 - (-dz + R(dz+zp,w0,labda,guess=guess_set))**2
    guess_rel = 0.01
    dz_sol = scipy.optimize.brentq(f_solve,0,zp*guess_rel)
    
    dz_calc = dz_sol+zp - Z
    phase = 2*np.pi*(dz_calc/labda)
    
    guess_course = R(rp,w0,labda)
    return dz_calc+zp

def w_new(x,y,z,w0,labda):
    dz = phase_extra([x,y,z],w0,labda) - z
    waist = w(z+dz,w0,labda)
    print(z+dz)
    return z+dz, waist
    

In [8]:
x=0.1
y=0.1
z = np.linalg.norm(Ndata.data.u_l_func_tot(1,t_vec[2]))
[z_1,w_1] = w_new(x,y,z,data.w0,data.labda)

2466524163.267166


In [9]:
def z_solve(x,y,z,calc_R=False):
    
    R_new = lambda z_tot: R(z_tot,w0,labda,guess=False)
    f_solve = lambda z_tot: (R_new(z_tot) - (R_new(z_tot)**2 - (x**2+y**2))**0.5) - (z_tot-z)
    z_sol = scipy.optimize.brentq(f_solve,z,z+1)
    if calc_R==True:
        return R(z_sol)
    else:
        return z_sol

In [10]:
def phasenoise_calc(Ndata,i,t,x,y,z,dx,dy,side='l',mode='phasenoise',sigmadzx=0,sigmadzy=0):
    labda = Ndata.data.labda
    if side=='l':
        z = np.linalg.norm(Ndata.data.u_l_func_tot(i,t))
    elif side=='r':
        z = np.linalg.norm(Ndata.data.u_r_func_tot(i,t))
    znew, wnew = w_new(x,y,z,Ndata.data.w0,Ndata.data.labda)
    
    
    u0 = np.exp(-((x**2+y**2)**2)/(wnew**2))
    
    print(u0,z,wnew)
    alphax = np.arcsin(x/z)
    alphay = np.arcsin(y/z)
    alpha = (alphax**2+alphay**2)**0.5
    w = Ndata.data.w0
    r = 0.01
    #alpha = 1e-6 # microrad
    beta = 0.25*np.pi
    r0 = 0.1
    k = (2*np.pi)/labda
    wac = r0/w
    q = 1.0/(wac**2) # ...In Sasso paper not with minus sign!!!

    z02=1.71
    z22abs = 12.6
    z13abs = 6.54
    z33abs = 5.46
    z04 = 3.53
    z11abs = alpha*k*r0*0.5
    th22 = -0.97
    th13 = 1.95
    th33 = -0.26
    th11=0
    
    A2 = (1+np.exp(q)+2*(1-np.exp(q))*(wac**2))/(1-np.exp(q))
    A4 = (1-np.exp(q)+6*(1+np.exp(q))*(wac**2) + 12*(1-np.exp(q))*(wac**4))/(1-np.exp(q))
    B = (-2*(1+3*(wac**2)+6*(wac**4)+6*(1-np.exp(q))*(wac**6)))/(1-np.exp(q))
    C = (-2*(1+5*(wac**2)+2*(7+2*np.exp(q))*(wac**4)+18*(1-np.exp(q))*(wac**6)))/(1-np.exp(q))
    D = (4*(np.exp(q)+6*np.exp(q)*(wac**2)-2*(2-np.exp(q) - np.exp(2*q))*(wac**4) - 12*((1-np.exp(q))**2)*(wac**6)))/((1-np.exp(q))**2)
    G = (24*((np.exp(q)*(wac**2))-(2-9*np.exp(q)+np.exp(2*q))*(wac**4) -2*(7-2*np.exp(q)-5*np.exp(2*q))*(wac**6) -30*(1-np.exp(q))*(wac**8)))/((1-np.exp(q))**2)
    
    #G = 0#3(24*(np.exp(q)*(wac**2)-(2-9*np.exp(q) + np.exp(2*q))*(wac**4) - 2*(7-2*np.exp(q) - 5*np.exp(2*q))*(wac**6) -30((1-np.exp(q))**2)*(wac**8)))/((1-np.exp(q))**2)
    E = (2*(np.exp(q) - ((1-np.exp(2*q))**2)*(wac**4)))/((1-np.exp(q))**2)
    F = (-1*(1+2*(wac**2)+2*(1-np.exp(q))*(wac**4)))/(1-np.exp(q))
    H = (6*(2*np.exp(q)*(wac**2) - (1-np.exp(2*q))*(wac**4)-4*((1-np.exp(q))**2)*(wac**6)))/((1-np.exp(q))**2)

    b0 = A2*z02+A4*z04
    b1 = B*z22abs*z33abs*np.cos(th33-th22-th11)+C*z22abs*z13abs*np.cos(th22-th13-th11)+D*z02*z13abs*np.cos(th13-th11)+G*z04*z13abs*np.cos(th13-th11)
    b2 = E*z02+F*z22abs*np.cos(th22-2*th11)+H*z04
    
    b00 = b0
    b10 = B*np.cos(th33-th22)*z33abs*z22abs +C*np.cos(th22-th13)*z13abs*z22abs+D*np.cos(th13)*z13abs*z02 +G*np.cos(th13)*z13abs*z22abs
    b01 = B*np.sin(th33-th22)*z33abs*z22abs +C*np.sin(th22-th13)*z13abs*z22abs+D*np.sin(th13)*z13abs*z02 +G*np.sin(th13)*z13abs*z22abs
    b20 = E*z02+F*np.cos(th22)*z22abs+H*z04
    b02 = E*z02-F*np.cos(th22)*z22abs+H*z04
    b11 = 2*F*np.sin(th22)*z22abs
        
    if mode == 'phasenoise':
        alphax = alphax*0
        alphay = alphay+0
        dzx = (k*r0*alphax)/2.0
        dzy = (k*r0*alphay)/2.0
        
        wnoise0z = b00+b10*dzx+b20*(dzx**2)+b01*dzy+b02*(dzy**2)+b11*dzx*dzy
    
    elif mode=='gaussian_profile':
        wnoise0z = b0+b1*z11abs+b2*(z11abs**2)
    
    a0 = (1.0/(np.pi*wnew**2))*dx*dy*u0
    a1 = (1.0/(np.pi*wnew**2))*dx*dy*wnoise0z*u0
    a2 = (-1.0/(2*np.pi*wnew**2))*dx*dy*(wnoise0z**2)*u0
    a3 = (-1.0/(6*np.pi*wnew**2))*dx*dy*(wnoise0z**3)*u0
    
    u0z = np.pi*(wnew**2)*(a0+1j*a1+a2+1j*a3)
    print(u0z)
    
    u0z_mag = (np.pi**2*wnew**4*abs(a0+1j*a1+a2)**2*k**2)/((2*np.pi*z)**2)
    
    
    #return np.exp(1j*wnoise0z)
    #return wnoise0z
    #u0z = u0*(1+1j*wnoise0z-0.5*(wnoise0z**2)-(1j/6)*(wnoise0z**3))
    
    #return [u0,wnoise0z,u0*np.exp(1j*wnoise0z)*((1j*k)/(2*np.pi*z)),np.exp(1j*wnoise0z)]
    #print(u0z,wnoise0z)
    return u0z_mag



In [11]:
def aperture(xlist,ylist,function):
    Nbins = len(xlist)
    D = max(xlist)
    ps = np.empty((Nbins,Nbins))
    for i in range(0,len(xlist)):
        for j in range(0,len(ylist)):
            r = (xlist[i]**2+ylist[j]**2)**0.5
            if r<=D:
                ps[i,j] = function(xlist[i],ylist[j])
            else:
                ps[i,j] = np.nan
    
    return ps

In [12]:
def phasenoise(i_self,t,Nbins,mode='phasenoise',side='l'):
    z = np.linalg.norm(Ndata.data.u_l_func_tot(i_self,t))
    D = Ndata.data.D
    xlist = np.linspace(-D,D,Nbins)
    ylist = xlist
    dx = xlist[1] - xlist[0]
    dy = ylist[1] - ylist[0]
    
    ps = np.empty((Nbins,Nbins))
    count=0
    inte = 0
    for i in range(0,len(xlist)):
        for j in range(0,len(ylist)):
            r = xlist[i]**2 + ylist[j]**2
            if r<=D**2:
                ps[i,j] = phasenoise_calc(Ndata,i_self,t,xlist[i],ylist[j],z,dx,dy,side=side,mode=mode)
                count=count+1
            else:
                ps[i,j] = np.nan
    
    return ps

In [13]:
LA = PAA_LISA.utils.la()

In [14]:
Nbins=10
D = Ndata.data.D
xlist = np.linspace(-D,D,Nbins)
ylist = xlist
Deltax = xlist[1]-xlist[0]
Deltay = ylist[1]-ylist[0]


In [15]:
### Gaussian beam
def phi_gauss(dx_list,dy_list,dX,dY,Ndata,i,t,side='l',jitter=False,tele_control='full control'):
    [i_self,i_left,i_right] = PAA_LISA.utils.i_slr(i)
    if side=='l':
        if tele_control=='full control':
            tele = Ndata.tele_l_fc(i,t)
        elif tele_control=='no control':
            tele = Ndata.tele_l(i,t)
    elif side=='r':
        if tele_control=='full control':
            tele = Ndata.tele_r_fc(i,t)
        elif tele_control=='no control':
            tele = Ndata.tele_r(i,t)
        
    vec = Ndata.data.u_l_func_tot(i,t)
    n = Ndata.data.n_func(i,t)
    [ang_x,ang_y] = LA.beam_ang(vec,tele,n)
    
    E0 = 1 #...adjust
    labda = Ndata.data.labda
    k = (2*np.pi)/labda
    w0 = Ndata.data.w0
    psi = 0 #Gouy angle
    if jitter==False:
        Zm = 0 #Exit pupil abberation
    else:
        Zm = jitter(t)
    #d = np.linalg.norm(vec)
    dX_ac = np.cos(ang_x)*dX
    dY_ac = np.cos(ang_y)*dY
    dZ_ac = np.sin(ang_x)*dX+np.sin(ang_y)*dY
    dR_ac = (dX_ac**2+dY_ac**2)**0.5
    d_ac = z_solve(dX_ac,dY_ac,np.linalg.norm(vec)+dZ_ac)
    #d_ac = d + dZ_ac
    
    D = max(dx_list)
    Deltax = dx_list[1]-dx_list[0]
    Deltay = dy_list[1]-dy_list[0]
    Nbinsx = len(dx_list)
    Nbinsy = len(dy_list)

    ps = np.empty((Nbinsx,Nbinsy),dtype = complex)
    for i in range(0,len(dx_list)):
        for j in range(0,len(dy_list)):
            
            dx = dx_list[i]
            dy = dy_list[j]
            dr = ((dx**2)+(dy**2))**0.5
            if dr<=D:
                s = (d_ac**2+(dR_ac-dr)**2)**0.5
                E = E0*np.exp((-(dr**2))/(w0**2))*np.exp(1j*psi)
                print(E,E0,dr,psi)
                #R_new = d_ac
                R_new = np.linalg.norm(vec)
                #print(np.exp(1j*((2*np.pi)/labda)*(Zm+s))/s)
                ps[i,j] = (E*np.exp(1j*((2*np.pi)/labda)*(Zm+s))/s)*Deltax*Deltay*np.exp((-1j*2*np.pi*R_new)/labda)
            else:
                ps[i,j] = np.nan
    
    diff_inte = np.nansum(ps)
    #diff_inte = diff_inte*np.exp(-1j*(2*np.pi*d)/labda)
    phi = np.angle(diff_inte)
    diff_inte_mag = (diff_inte.real**2+diff_inte.imag**2)**0.5
    #phi = np.log(diff_inte_new/diff_inte_mag).imag
        
    return diff_inte_mag,phi,ps

In [16]:
def calc_WF(i,t,rs = 'phi',tele_control='full control',side='l',jitter=False):
    I = lambda dx,dy: phi_gauss(xlist*A,ylist*A,dx,dy,Ndata,i,t,side=side,jitter=jitter,tele_control=tele_control)[0]
    ap_rec = lambda dx,dy: phi_gauss(xlist*A,ylist*A,dx,dy,Ndata,i,t,side=side,jitter=jitter,tele_control=tele_control)[1]
    phi = aperture(xlist,ylist,ap_rec)
    I = aperture(xlist,ylist,I)
    
    if rs=='phi':
        return phi
    elif rs=='I':
        return I



In [17]:
calc_WF(1,t_vec[2],tele_control='no control')

NameError: global name 'A' is not defined

In [ ]:
phi_gauss(xlist,ylist,0.1,0.1,Ndata,1,t_vec[2],side='l',jitter=False,tele_control='no control')[2]

In [ ]:
t=t_vec[2]
i=1
ap_rec = lambda dx,dy: phi_gauss(xlist,ylist,dx,dy,Ndata,i,t,side='l',jitter=False,tele_control='no control')[1]
phi = aperture(xlist,ylist,ap_rec)

In [ ]:
phi_t = phi_all(xlist,ylist,Ndata,1)

In [ ]:
phi

In [ ]:
A

In [ ]:
f,ax = plt.subplots(1,2)

p1 = ax[0].matshow(phi,cmap='magma',vmin=-np.pi,vmax=np.pi)
ax[0].set_title('Phi')
f.colorbar(p1, ax=ax[0])

p2 = ax[1].matshow(I,cmap='magma')
ax[1].set_title('I')
f.colorbar(p2, ax=ax[1])
#ax[1].colorbar()

In [ ]:
sigma_abb = 0.1*Ndata.data.labda
psd = lambda f: sigma_abb/f

In [ ]:
jitter = Ndata.Noise_time(0.0001,0.1,4095,psd,t_vec[-1])[1]

In [ ]:
phi

In [ ]:
data.L_rl_func_tot(2,12321)

In [ ]:
np.linalg.norm(data.u_l_func_tot(1,12321))

In [ ]:
np.linalg.norm(np.array(data.LISA.putp(1,12321)) - np.array(data.LISA.putp(2,12321)))*1e-9

In [ ]:
Ndata.tele_l_fc(1,12321)

In [ ]:
z_solve(0.1,0.1,)